In [1]:
!pip install selenium
!apt update
!apt install chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadco

In [41]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import selenium
import string
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import json
import re
from selenium import webdriver
from multiprocessing import Pool
import datetime
import fcntl
from pytz import timezone
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from itertools import repeat

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Collecting Player Data Links (1st Step)

In [ ]:
site_url = "https://www.pro-football-reference.com"

In [1]:
# grab index pages for all players
player_index_links = [f"{site_url}/players/{i}/" for i in string.ascii_uppercase]
player_index_links

NameError: name 'string' is not defined

In [ ]:
def get_webdriver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=options)

  # set implicit wait time
  driver.implicitly_wait(3) # seconds

  return driver

def get_player_info_urls(driver, base_url, index_urls):
  """
  for each url in index_urls:
    1. look for Div with id="div_players"
    2. Find all "a href" objects within "div_players" Div
      2.1 Keep track of the values associated with a href
  """
  all_player_links = []

  for url in index_urls:
    # go to index_url
    driver.get(url)

    # look for div_players div
    players_div = driver.find_element(By.ID, "div_players")

    # find all a hrefs
    a_hrefs = players_div.find_elements(By.TAG_NAME, 'a')

    print(f"Num Players On Page: {len(a_hrefs)}")

    player_links = [i.get_attribute("href") for i in a_hrefs]

    links_df = pd.DataFrame(player_links, columns=['link'])

    try:
      # read dataframe from memory
      saved_df = pd.read_csv("./test.csv")

      # concat newly found data with saved data
      links_df = pd.concat([saved_df, links_df]).drop_duplicates(['link'])
    except Exception as e:
      print(e)

    # save dataframe
    links_df.to_csv("./test.csv", index=False)

    print(links_df)

def get_active_player_info_urls(driver, base_url, index_urls, save_file, year_cutoff=2013):
  """
  for each url in index_urls:
    1. look for Div with id="div_players"
    2. Find all "a href" objects within "div_players" Div
      2.1 Keep track of the values associated with a href
  """
  all_player_links = []
  links_found = 0
  pattern = r"\d{4}[ ]*-[ ]*\d{4}"

  for url in index_urls:
    # go to index_url
    driver.get(url)

    # look for div_players div
    players_div = driver.find_element(By.ID, "div_players")

    # find all a hrefs
    a_hrefs = players_div.find_elements(By.TAG_NAME, 'a')

    links_found += len(a_hrefs)

    print(f"Num Player Links Found: {links_found}")

    # finding all p tags
    p_tags = players_div.find_elements(By.TAG_NAME, 'p')

    player_info = []

    for tag in p_tags:
      text = tag.text
      name = text.split("(")[0].strip()
      active_years_str = re.findall(pattern, text)[0]
      min_year, max_year = sorted([int(i.strip()) for i in active_years_str.split("-")])

      # print(f"name: {name}")
      # print(f"active_years_str: {active_years_str}")
      # print(f"years: {(min_year, max_year)}")

      if max_year >= year_cutoff:
        # print("Including Player")
        a_tag = tag.find_element(By.TAG_NAME, 'a')
        player_link = a_tag.get_attribute("href")
        # print(f"player_link: {player_link}")
        player_info.append((name, min_year, max_year, player_link))
        # links_df = pd.DataFrame(player_links, columns=['link'])

    links_df = pd.DataFrame(player_info, columns=['name', 'start_year', 'end_year', 'link'])

    try:
      # read dataframe from memory
      saved_df = pd.read_csv(save_file)

      # concat newly found data with saved data
      links_df = pd.concat([saved_df, links_df]).drop_duplicates(['link'])
    except Exception as e:
      print(e)

    # save dataframe
    links_df.to_csv(save_file, index=False)

    # print(links_df)

In [ ]:
# driver = get_webdriver()
# get_active_player_info_urls(
#   driver,
#   site_url,
#   player_index_links,
#   "/content/drive/MyDrive/SIADS_696/Data/active_player_links.csv"
# )
# get_player_info_urls(driver, site_url, player_index_links)

Num Player Links Found: 888
[Errno 2] No such file or directory: '/content/drive/MyDrive/SIADS_696/Data/active_player_links.csv'
Num Player Links Found: 3576
Num Player Links Found: 5525
Num Player Links Found: 6813
Num Player Links Found: 7331
Num Player Links Found: 8278
Num Player Links Found: 9613
Num Player Links Found: 11739
Num Player Links Found: 11875
Num Player Links Found: 13157
Num Player Links Found: 14115
Num Player Links Found: 15250
Num Player Links Found: 17860
Num Player Links Found: 18368
Num Player Links Found: 18798
Num Player Links Found: 20014
Num Player Links Found: 20057
Num Player Links Found: 21403
Num Player Links Found: 24012
Num Player Links Found: 25152
Num Player Links Found: 25225
Num Player Links Found: 25480
Num Player Links Found: 27511
Num Player Links Found: 27512
Num Player Links Found: 27662
Num Player Links Found: 27770


In [94]:
players_df = pd.read_csv("/content/drive/MyDrive/SIADS_696/Data/active_player_links.csv")
players_df

,name,start_year,end_year,link
0,Isaako Aaitui,2013,2013,https://www.pro-football-reference.com/players...
1,Israel Abanikanda,2023,2023,https://www.pro-football-reference.com/players...
2,Jared Abbrederis,2015,2017,https://www.pro-football-reference.com/players...
3,Mehdi Abdesmad,2016,2016,https://www.pro-football-reference.com/players...
4,Isa Abdul-Quddus,2011,2016,https://www.pro-football-reference.com/players...
...,...,...,...,...
6496,Greg Zuerlein,2012,2024,https://www.pro-football-reference.com/players...
6497,Jabari Zuniga,2020,2022,https://www.pro-football-reference.com/players...
6498,Jeremy Zuttah,2008,2016,https://www.pro-football-reference.com/players...
6499,Brandon Zylstra,2018,2022,https://www.pro-football-reference.com/players...


In [95]:
players_df.to_csv("/content/drive/MyDrive/SIADS_696/Data/players.csv")

# Scrape Data From Webpages (2nd Step)

In [15]:
def get_webdriver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=options)

  # set implicit wait time
  driver.implicitly_wait(3) # seconds

  return driver

def write_to_logfile(text):
  with open("/content/drive/MyDrive/SIADS_696/multiprocess_logfile.txt", "a+") as f:
    f.write(f"{text}\n")

def get_player_info_urls(driver, base_url, index_urls):
  """
  for each url in index_urls:
    1. look for Div with id="div_players"
    2. Find all "a href" objects within "div_players" Div
      2.1 Keep track of the values associated with a href
  """
  all_player_links = []

  for url in index_urls:
    # go to index_url
    driver.get(url)

    # look for div_players div
    players_div = driver.find_element(By.ID, "div_players")

    # find all a hrefs
    a_hrefs = players_div.find_elements(By.TAG_NAME, 'a')

    print(f"Num Players On Page: {len(a_hrefs)}")

    player_links = [i.get_attribute("href") for i in a_hrefs]

    links_df = pd.DataFrame(player_links, columns=['link'])

    try:
      # read dataframe from memory
      saved_df = pd.read_csv("./test.csv")

      # concat newly found data with saved data
      links_df = pd.concat([saved_df, links_df]).drop_duplicates(['link'])
    except Exception as e:
      print(e)

    # save dataframe
    links_df.to_csv("./test.csv", index=False)

    print(links_df)

def get_active_player_info_urls(driver, base_url, index_urls, save_file, year_cutoff=2013):
  """
  for each url in index_urls:
    1. look for Div with id="div_players"
    2. Find all "a href" objects within "div_players" Div
      2.1 Keep track of the values associated with a href
  """
  all_player_links = []
  links_found = 0
  pattern = r"\d{4}[ ]*-[ ]*\d{4}"

  for url in index_urls:
    # go to index_url
    driver.get(url)

    # look for div_players div
    players_div = driver.find_element(By.ID, "div_players")

    # find all a hrefs
    a_hrefs = players_div.find_elements(By.TAG_NAME, 'a')

    links_found += len(a_hrefs)

    print(f"Num Player Links Found: {links_found}")

    # finding all p tags
    p_tags = players_div.find_elements(By.TAG_NAME, 'p')

    player_info = []

    for tag in p_tags:
      text = tag.text
      name = text.split("(")[0].strip()
      active_years_str = re.findall(pattern, text)[0]
      min_year, max_year = sorted([int(i.strip()) for i in active_years_str.split("-")])

      # print(f"name: {name}")
      # print(f"active_years_str: {active_years_str}")
      # print(f"years: {(min_year, max_year)}")

      if max_year >= year_cutoff:
        # print("Including Player")
        a_tag = tag.find_element(By.TAG_NAME, 'a')
        player_link = a_tag.get_attribute("href")
        # print(f"player_link: {player_link}")
        player_info.append((name, min_year, max_year, player_link))
        # links_df = pd.DataFrame(player_links, columns=['link'])

    links_df = pd.DataFrame(player_info, columns=['name', 'start_year', 'end_year', 'link'])

    try:
      # read dataframe from memory
      saved_df = pd.read_csv(save_file)

      # concat newly found data with saved data
      links_df = pd.concat([saved_df, links_df]).drop_duplicates(['link'])
    except Exception as e:
      print(e)

    # save dataframe
    links_df.to_csv(save_file, index=False)

    # print(links_df)

def update_index(index_link, player_links, seen, include, min_year, max_year):
  with open(index_link, 'r') as f:
    # lock file for reading
    fcntl.flock(f.fileno(), fcntl.LOCK_EX)

    # reading
    df = pd.read_csv(f)

    # release lock on file
    fcntl.flock(f.fileno(), fcntl.LOCK_UN)

  for player_link in player_links:
    df.loc[df['link']==player_link, 'seen'] = seen
    df.loc[df['link']==player_link, 'include'] = include
    df.loc[df['link']==player_link, 'min_year'] = min_year
    df.loc[df['link']==player_link, 'max_year'] = max_year

  with open(index_link, 'w') as f:
    # lock file for writing
    fcntl.flock(f.fileno(), fcntl.LOCK_EX)

    df.to_csv(index_link, index=False)

    # release lock on file
    fcntl.flock(f.fileno(), fcntl.LOCK_UN)

def write_to_file(obj, filename):
  with open(filename, "a+") as f:
    # writing json obj to file
    f.write(obj)

def get_unseen_links(df, letter=None):
  if letter is None:
    temp_df = df[df['seen']==False]
  else:
    temp_df = df[(df['seen']==False) & (df['name'].str.split(" ").str[-1].str[0].str.upper()==letter)]

  print(temp_df)

  return temp_df['link'].tolist()

def repair_index(save=False):
  """
  Code to patch the index after it was overwritten
  """

  blank_index = pd.read_csv("/content/drive/MyDrive/SIADS_696/Data/active_player_links.csv")

  blank_index['seen'] = False
  blank_index['include'] = None
  blank_index['min_year'] = None
  blank_index['max_year'] = None

  stats_file = "/content/drive/MyDrive/SIADS_696/active_player_stats.json"

  with open(stats_file, "r") as f:
    lines = f.readlines()

    for line in lines:
      start_year=None
      end_year=None

      entry = json.loads(line)
      name = entry['name']
      data = entry['data']
      temp_df = blank_index[blank_index['name']==name]
      if len(temp_df) > 1:
        # print(f"Duplicate Names Found! {name}")
        for key in data.keys():
          data_df = pd.read_json(data[key])
          if "year" in [i.lower() for i in data_df.columns]:
            min_year = pd.to_numeric(data_df["year"], errors='coerce').min()
            max_year = pd.to_numeric(data_df["year"], errors='coerce').max()

            if start_year is None or min_year < start_year:
              start_year = min_year

            if end_year is None or max_year > end_year:
              end_year = max_year

        refined_temp_df = blank_index[(blank_index['name']==name) & (blank_index['start_year'].astype(int)==int(start_year)) & (blank_index['end_year'].astype(int)==int(end_year))].drop_duplicates(['name', 'start_year', 'end_year'])

        if len(refined_temp_df) == 1:
          blank_index.loc[(blank_index['name']==name) & (blank_index['start_year'].astype(int)==int(start_year)) & (blank_index['end_year'].astype(int)==int(end_year)), 'seen'] = True
        elif len(refined_temp_df) == 0:
          print(f"!!!FOUND NO MATCHING <name, start_year, end_year>: {name}!!!")
        elif len(refined_temp_df) > 1:
          print(f"!!!Duplicate name found <name, start_year, end_year>: {name}!!!")
          print(refined_temp_df.to_string())

      elif len(temp_df) == 1:
        blank_index.loc[blank_index['name']==name, 'seen'] = True
      else:
        print(f"FOUND NO MATCHING NAMES: {name}")
        print(f"Player Career: {start_year}-{end_year}")

    display(blank_index)
    print(blank_index.shape)

  if save:
    blank_index.to_csv("/content/drive/MyDrive/SIADS_696/active_player_links_index.csv", index=False)

def get_player_data_multiprocessing(index_df, num_links, year_cutoff=2013):
  print("Running get_player_data_multiprocessing()")

  # look for all_defense, all_receiving_and_rushing, all_rushing_and_receiving, all_passing, all_kicking, all_punting
  site_url = "https://www.pro-football-reference.com"

  driver = get_webdriver()

  tz = timezone('EST')
  current_time = datetime.datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')

  json_dfs = []

  max_retries = 3

  for i in range(num_links):
    retries = 0
    link = index_df.iloc[i]['link']

    while retries < max_retries:
      print("Calling driver.get() on next link")
      driver.get(link)

      print(f"Link: {link}")

      try:
        delay = 30
        info_div = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, 'info')))
        break
      except TimeoutException as e:
        print(f"Loading Took Too Much Time: {link}")
        retries += 1

    if retries == max_retries:
      print(f"Could Not Find Info Div at link: {link}")
      continue

    player_name = info_div.find_element(By.TAG_NAME, "h1").text
    info_paragraphs = info_div.find_elements(By.TAG_NAME, "p")
    earliest_data_year = None
    latest_data_year = None
    player_position = None

    for i in info_paragraphs:
      if "position:" in i.text.lower():
        player_position = i.text.lower().split("position:")[-1].strip().upper()

    all_tables = dict()

    # search for one of the desired divs
    for div_id in ['all_defense', 'all_receiving_and_rushing', 'all_rushing_and_receiving', 'all_passing', 'all_kicking', 'all_punting', 'all_ol_penalties', 'all_snap_counts']:
      try:
        data_div = driver.find_element(By.ID, div_id)
      except NoSuchElementException as e:
        print(f"Div ID Not Found: {div_id}")
        continue

      tables_html = data_div.find_elements(By.CLASS_NAME, "table_container")

      for table in tables_html:
        # Get the HTML of the table
        table_html = table.get_attribute('outerHTML')

        # Use Pandas to read the HTML table
        df = pd.read_html(table_html)[0]

        # print(f"{table.get_attribute('id')}\n{df}")

        if df.columns.nlevels > 1:
          # renaming unnamed columns
          for i, columns in enumerate(df.columns.levels):
            columns_new = columns.tolist()
            for j, row in enumerate(columns_new):
                if "Unnamed: " in row:
                    columns_new[j] = ""
            df = df.rename(columns=dict(zip(columns.tolist(), columns_new)), level=i)

          # flattening multiindex columns
          df.columns = ['_'.join(col).strip() for col in df.columns.values]
          df.columns = [i[1:].lower() if i[0]=="_" else i.lower() for i in df.columns]
        else:
          df.columns = [i.lower() for i in df.columns]

        if "year" in df.columns:
          df["year"] = df["year"].astype(str).str.replace("+", "")

          df["year"] = df["year"].astype(str).str.replace("*", "")

          min_year_in_table = pd.to_numeric(df["year"], errors='coerce').min()
          max_year_in_table = pd.to_numeric(df["year"], errors='coerce').max()

          if (earliest_data_year is None or np.isnan(earliest_data_year)) or min_year_in_table < earliest_data_year:
            earliest_data_year = min_year_in_table

          if (latest_data_year is None or np.isnan(latest_data_year)) or max_year_in_table > latest_data_year:
            latest_data_year = max_year_in_table

        data_name = div_id
        if "playoff" in table.get_attribute("id"): data_name += "_playoffs"

        all_tables[data_name] = df.to_json()

    if (len(all_tables.keys())==0) or ((latest_data_year is not None and not np.isnan(latest_data_year)) and latest_data_year >= year_cutoff):
      save_dict = {
          "name": player_name,
          "position": player_position,
          "data": all_tables,
          "link": link,
      }

      save_json = json.dumps(save_dict)+"\n"

      json_dfs.append(save_json)

      index_df.loc[index_df['link']==link, 'seen'] = True

    tz = timezone('EST')
    current_time = datetime.datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')
    print(f"[{current_time}] Finish Up {link}")

  return index_df, json_dfs

def split_index(index_file):
  df = pd.read_csv(index_file)
  seen_df = df[df['seen']==True]
  df = df[df['seen']==False]
  split_dfs = np.array_split(df, 2)

  for i, temp_df in enumerate(split_dfs):
    new_file = f"/content/drive/MyDrive/SIADS_696/active_player_links_index_split_{i+1}.csv"

#
def merge_splits():
  orig_df = pd.read_csv("/content/drive/MyDrive/SIADS_696/active_player_links_index.csv")
  orig_df = orig_df[orig_df['seen']==True]

  df1 = pd.read_csv("/content/drive/MyDrive/SIADS_696/active_player_links_index_split_1.csv")
  df2 = pd.read_csv("/content/drive/MyDrive/SIADS_696/active_player_links_index_split_2.csv")

  print(orig_df)
  print(df1)
  print(df2)

  merged_df = pd.concat([orig_df, df1, df2])

  print(f"Num Seen Players: {merged_df[merged_df['seen']==True].shape[0]}")
  print(f"Num Unseen Players: {merged_df[merged_df['seen']==False].shape[0]}")
  print(f"Total Players: {merged_df.shape[0]}")

  merged_df.to_csv("/content/drive/MyDrive/SIADS_696/active_player_links_index_after_merge.csv", index=False)

In [17]:

index_file = "/content/drive/MyDrive/SIADS_696/active_player_links_index_after_merge.csv"
df = pd.read_csv(index_file)

while df[df['seen']==False].shape[0] > 0:
  df = pd.read_csv(index_file)

  tz = timezone('EST')
  current_time = datetime.datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')
  print(f"[{current_time}] Num Seen So Far: {df[df['seen']==True].shape[0]}")

  num_processes = 1

  unseen_df = df[df['seen']==False].sample(frac=1).reset_index(drop=True)

  print(f"Num Unseen: {unseen_df.shape[0]}")

  split_dfs = np.array_split(unseen_df, num_processes)

  pool = Pool(processes=num_processes)  # Maximum number of browsers opened simultaneously

  save_file = "/content/drive/MyDrive/SIADS_696/active_player_stats.json"

  res = list(pool.apply_async(get_player_data_multiprocessing, args=(split_dfs[i], 1)) for i in range(num_processes))

  pool.close()
  pool.join()

  results = [i.get() for i in res]

  res_df = pd.concat([i[0] for i in results])

  print(f"res_df:\n{res_df}")

  json_dfs = []
  json_dfs = [i[1] for i in results]

  temp_df = pd.concat([df[df['seen']==True], res_df])

  tz = timezone('EST')
  current_time = datetime.datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')
  print(f"[{current_time}] Num Seen After Running Function: {temp_df[temp_df['seen']==True].shape[0]}")

  for json_df in json_dfs:
    if len(json_df) > 0:
      write_to_file(json_df[0], "/content/drive/MyDrive/SIADS_696/active_player_stats.json")

  temp_df.to_csv(index_file, index=False)

# Compile Scraped Data (3rd Step)